# Testing the size of the aggregated Hypercube

Used CP-Nets

In [2]:
# list of names to use
files = ["cpnet_n2c1d2_0000.xml", "cpnet_n2c1d2_0001.xml"]

# get n value
n = files[0][7:9]
if not n.isdigit():
    n = n[0]
n = int(n)    
    
# should we check whether n value is the same for all files??          

In [3]:
"""
genrates a list of the used variables
"""
def get_variables(n):
    vars = []
    for i in range(1,n+1):
        vars.append("x" + str(i))
    return vars 

variables = get_variables(n)

Helper functions for generating a data structure of the XML files

This only functions for binary domain cpnets!! (because the data structure simplifies the preference)

In [4]:
"""
forms a string representing the preference, e.g. x1=2  (from x1 and 2:1) 
meaning x1=2 is preferred over x1=1
"""
def reform_pref_string(variable, preference):
    new_string = variable + "=" + preference.split(":")[0]

"""
Input - ordered dict of a CP-Net formed by xmltodict
Output - a dictionary representing the rules
"""
def generating_rules(cpnet_dict):
    rules = {}
    for rule in cpnet_dict['PREFERENCE-SPECIFICATION']['PREFERENCE-STATEMENT']:
        variable_name = rule['PREFERENCE-VARIABLE'] #e.g. x1
        preference = rule['PREFERENCE'] #e.g. 2:1
        conditions = []
        if len(rule) > 3: # can be a list of conditions or a single item e.g. x2=2 or [x1=2, x2=2]
            conditions = rule['CONDITION']
            if type(rule['CONDITION']) is str:
                conditions = [conditions]
            
        cond_set = frozenset(conditions)
        pref = reform_pref_string(variable_name, preference)
        
        # add rule to the dictionary
        if cond_set in rules:
            rules[cond_set].append(pref)
        else:
            rules[cond_set] = [pref]
            
    return rules              

Reading XML files and generating a data structure which represent the rules of the CP-Net (a list of dictionaries)

In [5]:
import xmltodict

cp_list = []
for file in files:
    with open("./CPNets/" + file) as fd:
        doc = xmltodict.parse(fd.read())
        cp = generating_rules(doc)
        cp_list.append(cp)
               

Aggregating the CP-Nets to one hypercube CP-Net

In [11]:
"""
Generator for tuples of neighbours in the hypercube graph. Returns only positive variables.
If not included, the variable is negative.
TODO: Works though, but generates things double...
"""
def neighbours(seq_orig):
    seq = seq_orig.copy()
    for item in range(len(seq)):
        seq.pop(item)    
        yield (seq_orig, seq)
        yield from neighbours(seq)
        seq = seq_orig.copy()

#seq = ["x1", "x2", "x3", "x4"]        
#for i  in neighbours(seq):
#    print(i)

(['x1', 'x2', 'x3', 'x4'], ['x2', 'x3', 'x4'])
(['x2', 'x3', 'x4'], ['x3', 'x4'])
(['x3', 'x4'], ['x4'])
(['x4'], [])
(['x3', 'x4'], ['x3'])
(['x3'], [])
(['x2', 'x3', 'x4'], ['x2', 'x4'])
(['x2', 'x4'], ['x4'])
(['x4'], [])
(['x2', 'x4'], ['x2'])
(['x2'], [])
(['x2', 'x3', 'x4'], ['x2', 'x3'])
(['x2', 'x3'], ['x3'])
(['x3'], [])
(['x2', 'x3'], ['x2'])
(['x2'], [])
(['x1', 'x2', 'x3', 'x4'], ['x1', 'x3', 'x4'])
(['x1', 'x3', 'x4'], ['x3', 'x4'])
(['x3', 'x4'], ['x4'])
(['x4'], [])
(['x3', 'x4'], ['x3'])
(['x3'], [])
(['x1', 'x3', 'x4'], ['x1', 'x4'])
(['x1', 'x4'], ['x4'])
(['x4'], [])
(['x1', 'x4'], ['x1'])
(['x1'], [])
(['x1', 'x3', 'x4'], ['x1', 'x3'])
(['x1', 'x3'], ['x3'])
(['x3'], [])
(['x1', 'x3'], ['x1'])
(['x1'], [])
(['x1', 'x2', 'x3', 'x4'], ['x1', 'x2', 'x4'])
(['x1', 'x2', 'x4'], ['x2', 'x4'])
(['x2', 'x4'], ['x4'])
(['x4'], [])
(['x2', 'x4'], ['x2'])
(['x2'], [])
(['x1', 'x2', 'x4'], ['x1', 'x4'])
(['x1', 'x4'], ['x4'])
(['x4'], [])
(['x1', 'x4'], ['x1'])
(['x1'], [])
(['

In [7]:
"""
Helper function for formatting
Input: two neighbours as returned by neighbour function
Output: Tuple - first element the varying variable between the elements, second element set of constant variables
"""
def diff_neighbours(seq1, seq2):
    ones = set(seq1).intersection(seq2)
    varying = set(seq1).difference(seq2)
    twos = set(variables).difference(seq1)
    return (varying, set([i+"=1" for i in ones]).union([i+"=2" for i in twos]))


In [8]:
"""
For searching for matching rules in the dictionary (for a pair of neighbours in the hypercube)
"""
def matching_preferences(dictionary, premises):
    rules = []
    for key in dictionary:
        if key <= premises:
            rules.append(dictionary[key])
    return rules        

In [9]:
# iterate over each neighbouring pair 
for i in neighbours(variables):
    
    n1, n2 = i
    var, set_vars = diff_neighbours(n1, n2)
    
    # go through list of dictionaries
    for cpnet in cp_list:
        
        # search for all keys which are a subset of the set
        prefs = matching_preferences(cpnet, set_vars)
        
        # TODO: what do we do, if we have conflicting preferences?
    

#     collect corresponding rules from CP-Nets
#     apply majority rule


# simplify rule set
# measure number of rules
# measure also number of edges?? -> is that easily possible?